In [1]:
#Import the necessary libraries
import numpy as np
import pandas as pd
import difflib
from sklearn.feature_extraction.text import TfidfVectorizer
# TfidfVectorizer - This is used to convert text data into numerical values
from sklearn.metrics.pairwise import cosine_similarity
import os

In [2]:
#Change the working directory
os.chdir("D:\\Acmegrade ML Internship\\Assignments\\Projects\\Movie Recommendation")
display(os.getcwd())

'D:\\Acmegrade ML Internship\\Assignments\\Projects\\Movie Recommendation'

In [3]:
#Read and display the data
movies_data = pd.read_csv("movies.csv")
display(movies_data.head())

,index,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew,director
0,0,237000000,Action Adventure Fantasy Science Fiction,http://www.avatarmovie.com/,19995,culture clash future space war space colony so...,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,Sam Worthington Zoe Saldana Sigourney Weaver S...,"[{'name': 'Stephen E. Rivkin', 'gender': 0, 'd...",James Cameron
1,1,300000000,Adventure Fantasy Action,http://disney.go.com/disneypictures/pirates/,285,ocean drug abuse exotic island east india trad...,en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,...,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,Johnny Depp Orlando Bloom Keira Knightley Stel...,"[{'name': 'Dariusz Wolski', 'gender': 2, 'depa...",Gore Verbinski
2,2,245000000,Action Adventure Crime,http://www.sonypictures.com/movies/spectre/,206647,spy based on novel secret agent sequel mi6,en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,...,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,Daniel Craig Christoph Waltz L\u00e9a Seydoux ...,"[{'name': 'Thomas Newman', 'gender': 2, 'depar...",Sam Mendes
3,3,250000000,Action Crime Drama Thriller,http://www.thedarkknightrises.com/,49026,dc comics crime fighter terrorist secret ident...,en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,...,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,Christian Bale Michael Caine Gary Oldman Anne ...,"[{'name': 'Hans Zimmer', 'gender': 2, 'departm...",Christopher Nolan
4,4,260000000,Action Adventure Science Fiction,http://movies.disney.com/john-carter,49529,based on novel mars medallion space travel pri...,en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,...,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,Taylor Kitsch Lynn Collins Samantha Morton Wil...,"[{'name': 'Andrew Stanton', 'gender': 2, 'depa...",Andrew Stanton


In [4]:
#Display the shape of the movies_data.csv
display(movies_data.shape)

(4803, 24)

In [5]:
#Selecting only the relevant features for prediction
selected_features = ['genres','keywords','tagline','cast','director']
print(selected_features)

['genres', 'keywords', 'tagline', 'cast', 'director']


In [6]:
#Displaying the info of the movies_data.csv
display(movies_data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4803 entries, 0 to 4802
Data columns (total 24 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   index                 4803 non-null   int64  
 1   budget                4803 non-null   int64  
 2   genres                4775 non-null   object 
 3   homepage              1712 non-null   object 
 4   id                    4803 non-null   int64  
 5   keywords              4391 non-null   object 
 6   original_language     4803 non-null   object 
 7   original_title        4803 non-null   object 
 8   overview              4800 non-null   object 
 9   popularity            4803 non-null   float64
 10  production_companies  4803 non-null   object 
 11  production_countries  4803 non-null   object 
 12  release_date          4802 non-null   object 
 13  revenue               4803 non-null   int64  
 14  runtime               4801 non-null   float64
 15  spoken_languages     

None

In [7]:
#Check for null values
display(movies_data.isna().sum())

index                      0
budget                     0
genres                    28
homepage                3091
id                         0
keywords                 412
original_language          0
original_title             0
overview                   3
popularity                 0
production_companies       0
production_countries       0
release_date               1
revenue                    0
runtime                    2
spoken_languages           0
status                     0
tagline                  844
title                      0
vote_average               0
vote_count                 0
cast                      43
crew                       0
director                  30
dtype: int64

In [8]:
#Displaying the selected features
display(movies_data[selected_features].head())

,genres,keywords,tagline,cast,director
0,Action Adventure Fantasy Science Fiction,culture clash future space war space colony so...,Enter the World of Pandora.,Sam Worthington Zoe Saldana Sigourney Weaver S...,James Cameron
1,Adventure Fantasy Action,ocean drug abuse exotic island east india trad...,"At the end of the world, the adventure begins.",Johnny Depp Orlando Bloom Keira Knightley Stel...,Gore Verbinski
2,Action Adventure Crime,spy based on novel secret agent sequel mi6,A Plan No One Escapes,Daniel Craig Christoph Waltz L\u00e9a Seydoux ...,Sam Mendes
3,Action Crime Drama Thriller,dc comics crime fighter terrorist secret ident...,The Legend Ends,Christian Bale Michael Caine Gary Oldman Anne ...,Christopher Nolan
4,Action Adventure Science Fiction,based on novel mars medallion space travel pri...,"Lost in our world, found in another.",Taylor Kitsch Lynn Collins Samantha Morton Wil...,Andrew Stanton


In [9]:
#Checking the null values for the selected features alone
display(movies_data[selected_features].isna().sum())

genres       28
keywords    412
tagline     844
cast         43
director     30
dtype: int64

In [12]:
#Replacing null values with null strings and checking again
for feature in selected_features:
    movies_data[feature] = movies_data[feature].fillna('')
display(movies_data[selected_features].isna().sum())

genres      0
keywords    0
tagline     0
cast        0
director    0
dtype: int64

In [18]:
#Combining all the 5 selected features
combined_features = movies_data['genres']+' '+movies_data['keywords']+' '+movies_data['tagline']+' '+movies_data['cast']+' '+movies_data['director']
display (combined_features)

0       Action Adventure Fantasy Science Fiction cultu...
1       Adventure Fantasy Action ocean drug abuse exot...
2       Action Adventure Crime spy based on novel secr...
3       Action Crime Drama Thriller dc comics crime fi...
4       Action Adventure Science Fiction based on nove...
                              ...                        
4798    Action Crime Thriller united states\u2013mexic...
4799    Comedy Romance  A newlywed couple's honeymoon ...
4800    Comedy Drama Romance TV Movie date love at fir...
4801      A New Yorker in Shanghai Daniel Henney Eliza...
4802    Documentary obsession camcorder crush dream gi...
Length: 4803, dtype: object

In [28]:
#All words will be converted into their equivalent numbers by TfidfVectorizer
vectorizer = TfidfVectorizer()
feature_vectors = vectorizer.fit_transform(combined_features)
display (feature_vectors.shape)
display(pd.DataFrame(feature_vectors.toarray()))


(4803, 17318)

,0,1,2,3,4,5,6,7,8,9,...,17308,17309,17310,17311,17312,17313,17314,17315,17316,17317
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4798,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4799,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4800,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4801,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [29]:
print(feature_vectors)

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 124266 stored elements and shape (4803, 17318)>
  Coords	Values
  (0, 201)	0.07860022416510505
  (0, 274)	0.09021200873707368
  (0, 5274)	0.11108562744414445
  (0, 13599)	0.1036413987316636
  (0, 5437)	0.1036413987316636
  (0, 3678)	0.21392179219912877
  (0, 3065)	0.22208377802661425
  (0, 5836)	0.1646750903586285
  (0, 14378)	0.33962752210959823
  (0, 16587)	0.12549432354918996
  (0, 3225)	0.24960162956997736
  (0, 14271)	0.21392179219912877
  (0, 4945)	0.24025852494110758
  (0, 15261)	0.07095833561276566
  (0, 16998)	0.1282126322850579
  (0, 11192)	0.09049319826481456
  (0, 11503)	0.27211310056983656
  (0, 13349)	0.15021264094167086
  (0, 17007)	0.23643326319898797
  (0, 17290)	0.20197912553916567
  (0, 13319)	0.2177470539412484
  (0, 14064)	0.20596090415084142
  (0, 16668)	0.19843263965100372
  (0, 14608)	0.15150672398763912
  (0, 8756)	0.22709015857011816
  :	:
  (4801, 403)	0.17727585190343229
  (4801, 4835)	0.247137650

In [30]:
#Getting the similarity scores using cosine similarity
similarity = cosine_similarity(feature_vectors)
print  (similarity)


[[1.         0.07219487 0.037733   ... 0.         0.         0.        ]
 [0.07219487 1.         0.03281499 ... 0.03575545 0.         0.        ]
 [0.037733   0.03281499 1.         ... 0.         0.05389661 0.        ]
 ...
 [0.         0.03575545 0.         ... 1.         0.         0.02651502]
 [0.         0.         0.05389661 ... 0.         1.         0.        ]
 [0.         0.         0.         ... 0.02651502 0.         1.        ]]


In [31]:
#Print Shape Cosine Similarity
display(similarity.shape)

(4803, 4803)

In [37]:
#Creating a list with all the movie names given in the dataset
list_of_all_titles = movies_data['title'].tolist()
print(list_of_all_titles)

#Length of the List 
print (len(list_of_all_titles))

['Avatar', "Pirates of the Caribbean: At World's End", 'Spectre', 'The Dark Knight Rises', 'John Carter', 'Spider-Man 3', 'Tangled', 'Avengers: Age of Ultron', 'Harry Potter and the Half-Blood Prince', 'Batman v Superman: Dawn of Justice', 'Superman Returns', 'Quantum of Solace', "Pirates of the Caribbean: Dead Man's Chest", 'The Lone Ranger', 'Man of Steel', 'The Chronicles of Narnia: Prince Caspian', 'The Avengers', 'Pirates of the Caribbean: On Stranger Tides', 'Men in Black 3', 'The Hobbit: The Battle of the Five Armies', 'The Amazing Spider-Man', 'Robin Hood', 'The Hobbit: The Desolation of Smaug', 'The Golden Compass', 'King Kong', 'Titanic', 'Captain America: Civil War', 'Battleship', 'Jurassic World', 'Skyfall', 'Spider-Man 2', 'Iron Man 3', 'Alice in Wonderland', 'X-Men: The Last Stand', 'Monsters University', 'Transformers: Revenge of the Fallen', 'Transformers: Age of Extinction', 'Oz: The Great and Powerful', 'The Amazing Spider-Man 2', 'TRON: Legacy', 'Cars 2', 'Green Lant

In [42]:
#Finding the close match for the movie name given by the user (only finds close matches based on the movie title comparision, not by the feature_vectors)
movie_name = input(' Enter your favourite movie name : ')
find_close_match = difflib.get_close_matches(movie_name, list_of_all_titles)
print(find_close_match)

#Display the close match 
closest_match = find_close_match[0]
print(closest_match)

 Enter your favourite movie name :  titanic


['Titanic', 'Maniac', 'Hitman']
Titanic


In [43]:
#Finding the index of the movie with the title
index_of_the_movie = movies_data[movies_data.title == closest_match]['index'].values[0]
print(index_of_the_movie)


25


In [44]:
#Get the similarity row for the selected index 
#These are the similarity values for the movie entered by the user 
similarity_score = list(enumerate(similarity[index_of_the_movie]))
print(similarity_score)


[(0, np.float64(0.04284926095950226)), (1, np.float64(0.0)), (2, np.float64(0.011467364105495698)), (3, np.float64(0.010046067299207016)), (4, np.float64(0.010913900929666012)), (5, np.float64(0.013271868195005247)), (6, np.float64(0.0)), (7, np.float64(0.04427635120584794)), (8, np.float64(0.0)), (9, np.float64(0.01105296767401108)), (10, np.float64(0.04077473745704553)), (11, np.float64(0.005154880168301485)), (12, np.float64(0.0)), (13, np.float64(0.011810642378201053)), (14, np.float64(0.011135108727728623)), (15, np.float64(0.010700980101090706)), (16, np.float64(0.010727286350311041)), (17, np.float64(0.0)), (18, np.float64(0.0)), (19, np.float64(0.058321349267840134)), (20, np.float64(0.0)), (21, np.float64(0.0)), (22, np.float64(0.0)), (23, np.float64(0.0)), (24, np.float64(0.0030136767341105873)), (25, np.float64(1.0000000000000004)), (26, np.float64(0.0)), (27, np.float64(0.035917907722692974)), (28, np.float64(0.005803448123074126)), (29, np.float64(0.017798015596744543)), (

In [45]:
#Length of Similarity Score 
len(similarity_score)


4803

In [47]:
#Sorting the movies based on their similarity score
sorted_similar_movies = sorted(similarity_score, key = lambda x:x[1], reverse = True) 
display(sorted_similar_movies)


[(25, np.float64(1.0000000000000004)),
 (593, np.float64(0.2300727107211525)),
 (454, np.float64(0.16589907730438733)),
 (1081, np.float64(0.16035128823311431)),
 (2011, np.float64(0.13803443166572782)),
 (3631, np.float64(0.1333839815786052)),
 (4133, np.float64(0.13183140853614747)),
 (656, np.float64(0.12606246128803086)),
 (1985, np.float64(0.12349759621747017)),
 (2036, np.float64(0.12348320550112096)),
 (316, np.float64(0.11879207685844573)),
 (142, np.float64(0.11838847957751894)),
 (2449, np.float64(0.11814094328522082)),
 (846, np.float64(0.11610782635112268)),
 (49, np.float64(0.11503079796465221)),
 (765, np.float64(0.1136042644493443)),
 (439, np.float64(0.11317975752102441)),
 (2245, np.float64(0.11276113457467152)),
 (720, np.float64(0.10752741389011594)),
 (1362, np.float64(0.10696934437645386)),
 (1629, np.float64(0.10631964670577465)),
 (1500, np.float64(0.10616709183973469)),
 (972, np.float64(0.10565080846258226)),
 (622, np.float64(0.10374165072398074)),
 (2955, np.

In [48]:
#Prints the first 30 movies that have high similarity score
print('Movies suggested for you : \n')
i = 1
for movie in sorted_similar_movies:
  index = movie[0]
  title_from_index = movies_data[movies_data.index==index]['title'].values[0]
  if (i<30):
    print(i, '.',title_from_index)
    i+=1


Movies suggested for you : 

1 . Titanic
2 . The Dilemma
3 . The Day the Earth Stood Still
4 . Revolutionary Road
5 . Cheri
6 . Heavenly Creatures
7 . Plush
8 . Primary Colors
9 . The Bridge of San Luis Rey
10 . Saving Silverman
11 . Gangs of New York
12 . Flushed Away
13 . Sense and Sensibility
14 . Stuck on You
15 . The Great Gatsby
16 . Almost Famous
17 . Shutter Island
18 . The Brothers Bloom
19 . Contagion
20 . The Blind Side
21 . Little Black Book
22 . This Is the End
23 . The Host
24 . Body of Lies
25 . Romance & Cigarettes
26 . Ladyhawke
27 . The Aviator
28 . The Believer
29 . The Phantom


In [65]:
#Consolidated code for the recommendation of first 30 movies based on the similarity score 
movie_name = input(' Enter your favourite movie name : ')
list_of_all_titles = movies_data['title'].tolist()
find_close_match = difflib.get_close_matches(movie_name, list_of_all_titles)
closest_match = find_close_match[0]
index_of_the_movie = movies_data[movies_data.title == closest_match]['index'].values[0]
similarity_score = list(enumerate(similarity[index_of_the_movie]))
sorted_similar_movies = sorted(similarity_score, key = lambda x:x[1], reverse = True) 

print('Movies suggested for you : \n')
i = 1

for movie in sorted_similar_movies:
  index = movie[0]
  title_from_index = movies_data[movies_data.index==index]['title'].values[0]
  if (i<31):
    print(i, '.',title_from_index)
    i+=1


 Enter your favourite movie name :  dark knight rises


Movies suggested for you : 

1 . The Dark Knight Rises
2 . Batman Begins
3 . The Dark Knight
4 . The Prestige
5 . Batman Returns
6 . Interstellar
7 . Batman & Robin
8 . Batman
9 . Inception
10 . Superman
11 . Legend
12 . Kick-Ass 2
13 . Kick-Ass
14 . Child 44
15 . Get Smart
16 . The Killer Inside Me
17 . RockNRolla
18 . Watchmen
19 . Suicide Squad
20 . True Romance
21 . Bronson
22 . The Other Side of Heaven
23 . Superman II
24 . Superman III
25 . Jonah Hex
26 . Batman Forever
27 . Insomnia
28 . Captain Corelli's Mandolin
29 . Catwoman
30 . The Contender
